In [2]:
import pandas as pd

In [37]:
census2011prov = pd.read_csv("census-2011-2016-boundaries-pr-age-population.csv", skiprows=9, dtype={"Age in completed years": str})
census2011localmun = pd.read_csv("census-2011-2016-boundaries-mn-age-population.csv", skiprows=9, dtype={"Age in completed years": str})
census2011districtmun = pd.read_csv("census-2011-2016-boundaries-dc-age-population.csv", skiprows=9, dtype={"Age in completed years": str})
census2011 = pd.concat([census2011prov, census2011districtmun, census2011localmun])

In [38]:
census2011 = census2011.drop(columns=["Summation Options", "Unnamed: 4"])

In [47]:
census2011[5000:]

,Geography 2016,Age in completed years,Count
3901,DC28,119,1.12287
3902,DC28,120,0.00000
3903,DC28,Unspecified,0.00000
3904,DC29,0,15280.07153
3905,DC29,1,15343.76226
...,...,...,...
25982,LIM476 : Greater Tubatse/Fetakgomo,118,0.00000
25983,LIM476 : Greater Tubatse/Fetakgomo,119,0.00000
25984,LIM476 : Greater Tubatse/Fetakgomo,120,0.00000
25985,LIM476 : Greater Tubatse/Fetakgomo,Unspecified,0.00000


In [23]:
commsurv2016prov = pd.read_csv("community-survey-2016-pr-age-population.csv", skiprows=9, dtype={"Age": str})
commsurv2016mun = pd.read_csv("community-survey-2016-dc-mn-age-population.csv", skiprows=9, dtype={"Age": str})
commsurv2016 = pd.concat([commsurv2016prov, commsurv2016mun])

In [24]:
commsurv2016 = commsurv2016.drop(columns=["Summation Options", "Unnamed: 4"])

,Age,Geography hierarchy 2016,Count
0,0,Western Cape,112560.644948
1,0,Eastern Cape,144153.659291
2,0,Northern Cape,22959.908748
3,0,Free State,51272.282969
4,0,KwaZulu-Natal,243054.420864
...,...,...,...
6197,113,Total,68.551399
6198,114,Total,87.850892
6199,115,Total,219.438187
6200,116,Total,100.469795


In [36]:
commsurv2016[6000:]

,Age,Geography hierarchy 2016,Count
4946,32,JHB,85191.470848
4947,33,JHB,86466.518837
4948,34,JHB,71136.594468
4949,35,JHB,111067.947836
4950,36,JHB,104687.048422
...,...,...,...
6197,113,Total,68.551399
6198,114,Total,87.850892
6199,115,Total,219.438187
6200,116,Total,100.469795


In [47]:
census2011.merge(commsurv2016, how="outer", left_on=["Geography 2016", "Age in completed years"], right_on=["Geography hierarchy 2016", "Age"], suffixes=("_2011", "_2016"))

,Geography 2016,Age in completed years,Count_2011,Unnamed: 4_2011,Geography hierarchy 2016,Age,Count_2016,Unnamed: 4_2016
0,Western Cape,0,117650.51541,NaN,Western Cape,0,112560.644948,NaN
1,Western Cape,1,114638.43418,NaN,Western Cape,1,112426.124685,NaN
2,Western Cape,2,115387.78545,NaN,Western Cape,2,121214.976259,NaN
3,Western Cape,3,110466.22291,NaN,Western Cape,3,110598.166595,NaN
4,Western Cape,4,106657.16689,NaN,Western Cape,4,113570.907357,NaN
...,...,...,...,...,...,...,...,...
39508,NaN,NaN,NaN,NaN,DC47: Sekhukhune,112,8.043213,NaN
39509,NaN,NaN,NaN,NaN,DC47: Sekhukhune,113,0.000000,NaN
39510,NaN,NaN,NaN,NaN,DC47: Sekhukhune,114,9.493638,NaN
39511,NaN,NaN,NaN,NaN,DC47: Sekhukhune,115,8.487667,NaN


In [26]:
census2011 = census2011[census2011["Geography 2016"] != "Total"]

In [27]:
census2011

,Geography 2016,Age in completed years,Count
0,Western Cape,0,117650.51541
1,Western Cape,1,114638.43418
2,Western Cape,2,115387.78545
3,Western Cape,3,110466.22291
4,Western Cape,4,106657.16689
...,...,...,...
32326,9027,118,0.00000
32327,9027,119,0.00000
32328,9027,120,0.00000
32329,9027,Unspecified,0.00000
